In [1]:
import pandas as pd
import classification_algorithms
import json

### Credit scoring - Monthly cash balances (did not yield good result)

In [2]:
monthly_cash_data = pd.read_csv('monthly_cash_balance.csv')

In [3]:
monthly_cash_data = monthly_cash_data.drop(columns='Unnamed: 0')
monthly_cash_data.head()

,Respodent,Year,Month,Cash,Cash Change,date,Firm ID,Class
0,4425.0,2021,2,-200000.0,0.0,2021-02-28,DD-ET-1930,Strong
1,4425.0,2021,3,-200000.0,0.0,2021-03-31,DD-ET-1930,Strong
2,4425.0,2021,4,-193281.0,6719.0,2021-04-30,DD-ET-1930,Strong
3,4425.0,2021,5,-193177.0,104.0,2021-05-31,DD-ET-1930,Strong
4,4425.0,2021,6,-126689.0,66488.0,2021-06-30,DD-ET-1930,Strong


In [4]:
demographic_data = pd.read_excel(r'lift_data\Income Report - Mon Feb 13 2023.xlsx', sheet_name='Demographics')

In [5]:
demographic_data.head()

,Respondent ID,Gender,Age,Number of children,Marital Status,Country of Residence,Citizenship,Firm ID,Sector type,Date firm established,Age of Firm,Number of Owners,Owner/s Gender,Number of Employees,Industry,Location in the Country,Activation Date
0,4425,Male,35,3,Married,Ethiopia,Ethiopia,DD-ET-1930,AD,10/ 30/2016,7 - 10 years,1,All men,4,Other,Diredawa,2021-03-29
1,4429,Male,32,0,Single,Ethiopia,Ethiopia,AA-KG-SA-1376,EE-01,dd/03/2015,5 - 6 years,2,All men,18,Light Manu,Addis Ababa,2021-03-29
2,4431,Male,27,0,Single,Ethiopia,Ethiopia,AA-KG-SA-1495,Garment production,2018/03/22,2 years,4 or more,Both men and women,25,Light Manu,Addis Ababa,2021-03-29
3,4432,Male,32,2,Married,Ethiopia,Ethiopia,DD-ET-2194,U-03,0,5 - 6 years,1,All men,5,Other,Diredawa,2021-03-29
4,4433,Female,61,1,Widowed,Ethiopia,Ethiopia,HR-DE-2171,X,08/07/2017,3 - 4 years,3,Both men and women,12,Other,Harar,2021-03-29


In [6]:
demographic_data.columns

Index(['Respondent ID', 'Gender', 'Age', 'Number of children',
       'Marital Status', 'Country of Residence', 'Citizenship', 'Firm ID',
       'Sector type', 'Date firm established', 'Age of Firm',
       'Number of Owners', 'Owner/s Gender', 'Number of Employees', 'Industry',
       'Location in the Country', 'Activation Date'],
      dtype='object')

In [7]:
demo_cols = [col for col in demographic_data.columns if col not in ['Activation Date', 'Date firm established', 'Firm ID', 'Age of Firm']]
demo_cols

['Respondent ID',
 'Gender',
 'Age',
 'Number of children',
 'Marital Status',
 'Country of Residence',
 'Citizenship',
 'Sector type',
 'Number of Owners',
 'Owner/s Gender',
 'Number of Employees',
 'Industry',
 'Location in the Country']

In [8]:
main_demo_data = demographic_data[demo_cols]
main_demo_data.head()

,Respondent ID,Gender,Age,Number of children,Marital Status,Country of Residence,Citizenship,Sector type,Number of Owners,Owner/s Gender,Number of Employees,Industry,Location in the Country
0,4425,Male,35,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
1,4429,Male,32,0,Single,Ethiopia,Ethiopia,EE-01,2,All men,18,Light Manu,Addis Ababa
2,4431,Male,27,0,Single,Ethiopia,Ethiopia,Garment production,4 or more,Both men and women,25,Light Manu,Addis Ababa
3,4432,Male,32,2,Married,Ethiopia,Ethiopia,U-03,1,All men,5,Other,Diredawa
4,4433,Female,61,1,Widowed,Ethiopia,Ethiopia,X,3,Both men and women,12,Other,Harar


In [9]:
main_demo_data.rename(columns={'Respondent ID': 'Respondent'}, inplace=True)
main_demo_data.head()

C:\Users\DIO\AppData\Local\Temp\ipykernel_9936\448913578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_demo_data.rename(columns={'Respondent ID': 'Respondent'}, inplace=True)


,Respondent,Gender,Age,Number of children,Marital Status,Country of Residence,Citizenship,Sector type,Number of Owners,Owner/s Gender,Number of Employees,Industry,Location in the Country
0,4425,Male,35,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
1,4429,Male,32,0,Single,Ethiopia,Ethiopia,EE-01,2,All men,18,Light Manu,Addis Ababa
2,4431,Male,27,0,Single,Ethiopia,Ethiopia,Garment production,4 or more,Both men and women,25,Light Manu,Addis Ababa
3,4432,Male,32,2,Married,Ethiopia,Ethiopia,U-03,1,All men,5,Other,Diredawa
4,4433,Female,61,1,Widowed,Ethiopia,Ethiopia,X,3,Both men and women,12,Other,Harar


In [10]:
res_class_map = {}
for i, row in monthly_cash_data.iterrows():
    res_class_map[row['Respodent']] = row['Class']
res_class_map

{4425.0: 'Strong',
 4429.0: 'Strong',
 4431.0: 'Poor',
 4432.0: 'Poor',
 4433.0: 'Medium',
 4435.0: 'Strong',
 4436.0: 'Medium',
 4437.0: 'Strong',
 4438.0: 'Strong',
 4440.0: 'Strong',
 4442.0: 'Strong',
 4443.0: 'Strong',
 4444.0: 'Strong',
 4445.0: 'Strong',
 4446.0: 'Medium',
 4448.0: 'Strong',
 4450.0: 'Poor',
 4451.0: 'Strong',
 4452.0: 'Weak',
 4453.0: 'Medium',
 4456.0: 'Medium',
 4457.0: 'Strong',
 4458.0: 'Strong',
 4459.0: 'Medium',
 4460.0: 'Strong',
 4462.0: 'Poor',
 4463.0: 'Medium',
 4464.0: 'Medium',
 4465.0: 'Poor',
 4466.0: 'Strong',
 4467.0: 'Poor',
 4468.0: 'Strong',
 4469.0: 'Medium',
 4470.0: 'Strong',
 4471.0: 'Medium',
 4472.0: 'Weak',
 4473.0: 'Strong',
 4474.0: 'Poor',
 4475.0: 'Strong',
 4477.0: 'Strong',
 4478.0: 'Poor',
 4480.0: 'Strong',
 4481.0: 'Medium',
 4482.0: 'Poor',
 4483.0: 'Medium',
 4484.0: 'Strong',
 4485.0: 'Poor',
 4487.0: 'Medium',
 4490.0: 'Medium',
 4491.0: 'Medium',
 4493.0: 'Medium',
 4494.0: 'Medium',
 4495.0: 'Poor',
 4496.0: 'Strong',


In [11]:
average_cash_data = monthly_cash_data[['Respodent', 'Cash', 'Cash Change']].groupby(by='Respodent').mean().reset_index()
average_cash_data.head()

,Respodent,Cash,Cash Change
0,4425.0,-3964.058824,30669.970588
1,4429.0,760267.214286,88948.814286
2,4431.0,40160.000000,95995.000000
3,4432.0,1080.000000,0.000000
4,4433.0,268211.294615,31847.886923


In [12]:
class_data = []
for i, row in average_cash_data.iterrows():
    class_data.append(res_class_map[row['Respodent']])


In [13]:
average_cash_data['Class'] = class_data
average_cash_data.head()

,Respodent,Cash,Cash Change,Class
0,4425.0,-3964.058824,30669.970588,Strong
1,4429.0,760267.214286,88948.814286,Strong
2,4431.0,40160.000000,95995.000000,Poor
3,4432.0,1080.000000,0.000000,Poor
4,4433.0,268211.294615,31847.886923,Medium


In [14]:
rand_forest = classification_algorithms.RandForest(average_cash_data, ['Cash'], 'Class',42)

In [15]:
rand_forest.summarize()

{0: 'Strong', 1: 'Poor', 2: 'Medium', 3: 'Weak'}


{'confusion_matrix':     Class  Strong  Poor  Medium  Weak  precision
 0  Strong       8     2       3     3   0.500000
 1    Poor       2     5       0     0   0.714286
 2  Medium       6     1       4     3   0.285714
 3    Weak       1     0       1     1   0.333333,
 'accuracy': 0.45}

In [16]:
classification_algorithms.pick_classification_model(average_cash_data, ['Cash'], 'Class', 42)

Training LogisticML model with the data...
=> Calculating cross validation score...
Training KNN model with the data...
=> Calculating cross validation score...
Training Baysian model with the data...
=> Calculating cross validation score...
Training SVMML model with the data...
=> Calculating cross validation score...
Training RandForest model with the data...
=> Calculating cross validation score...
        model     score
0     Baysian  0.415217
1         KNN  0.356522
2  RandForest  0.349638
3  LogisticML  0.330435
4       SVMML  0.305435
------Best is Baysian-------


In [17]:
len(monthly_cash_data['Respodent'].unique())

158

### Removing firms that had negative cash balances at any point

In [18]:
monthly_cash_data.head()

,Respodent,Year,Month,Cash,Cash Change,date,Firm ID,Class
0,4425.0,2021,2,-200000.0,0.0,2021-02-28,DD-ET-1930,Strong
1,4425.0,2021,3,-200000.0,0.0,2021-03-31,DD-ET-1930,Strong
2,4425.0,2021,4,-193281.0,6719.0,2021-04-30,DD-ET-1930,Strong
3,4425.0,2021,5,-193177.0,104.0,2021-05-31,DD-ET-1930,Strong
4,4425.0,2021,6,-126689.0,66488.0,2021-06-30,DD-ET-1930,Strong


In [19]:
to_remove = set()
for i, row in monthly_cash_data.iterrows():
    if row['Cash'] < 0:
        to_remove.add(row['Respodent'])

len(to_remove) 

107

In [20]:
remaining_firm_list = set(monthly_cash_data['Respodent'].unique()).difference(to_remove)
len(remaining_firm_list)

51

In [21]:
remaining_firm_df = monthly_cash_data[monthly_cash_data['Respodent'].isin(remaining_firm_list)].reset_index().drop(columns='index')
remaining_firm_df

,Respodent,Year,Month,Cash,Cash Change,date,Firm ID,Class
0,4432.0,2021,4,1080.0,0.0,2021-04-30,DD-ET-2194,Poor
1,4451.0,2021,4,19000.0,0.0,2021-04-30,AD-CN-KE-1684,Strong
2,4451.0,2021,5,53650.0,34650.0,2021-05-31,AD-CN-KE-1684,Strong
3,4451.0,2021,6,234650.0,181000.0,2021-06-30,AD-CN-KE-1684,Strong
4,4451.0,2021,7,305250.0,70600.0,2021-07-31,AD-CN-KE-1684,Strong
...,...,...,...,...,...,...,...,...
572,5104.0,2022,3,435467.0,2700.0,2022-03-31,AA-BA-HE-1111,Weak
573,5104.0,2022,4,439567.0,4100.0,2022-04-30,AA-BA-HE-1111,Weak
574,5104.0,2022,10,439937.0,370.0,2022-10-31,AA-BA-HE-1111,Weak
575,4759.0,2021,4,0.0,0.0,2021-04-30,AA-KE-SA-1331,Poor


In [22]:
remaining_average_df = remaining_firm_df[['Respodent', 'Cash', 'Class', 'Cash Change']].groupby(by=['Respodent', 'Class']).mean().reset_index()
remaining_average_df.head()

,Respodent,Class,Cash,Cash Change
0,4432.0,Poor,1080.000000,0.000000
1,4451.0,Strong,520715.384615,69688.461538
2,4460.0,Strong,295112.538462,37557.384615
3,4462.0,Poor,688059.351538,82923.151538
4,4468.0,Strong,389106.153846,40951.538462


In [23]:
remain_rand_forest = classification_algorithms.RandForest(remaining_average_df, ['Cash Change'], 'Class', 50)

In [24]:
remain_rand_forest.summarize()

{0: 'Poor', 1: 'Strong', 2: 'Weak', 3: 'Medium'}


{'confusion_matrix':     Class  Poor  Strong  Medium  precision
 0    Poor     4       0       1   0.800000
 1  Strong     0       2       1   0.666667
 2  Medium     0       2       1   0.333333,
 'accuracy': 0.5384615384615384}

In [25]:
classification_algorithms.pick_classification_model(remaining_average_df, ['Cash Change'], 'Class')

Training LogisticML model with the data...
=> Calculating cross validation score...
Training KNN model with the data...
=> Calculating cross validation score...
Training Baysian model with the data...
=> Calculating cross validation score...
Training SVMML model with the data...


c:\Users\DIO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\DIO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\DIO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\DIO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


=> Calculating cross validation score...
Training RandForest model with the data...
=> Calculating cross validation score...


c:\Users\DIO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


        model     score
0  LogisticML  0.553571
1       SVMML  0.528571
2     Baysian  0.525000
3         KNN  0.478571
4  RandForest  0.400000
------Best is LogisticML-------


### Trying with other financial figures (this worked much better)

In [26]:
all_monthly_financials = pd.read_csv('all_monthly_financials.csv')

all_monthly_financials.drop(columns='Unnamed: 0', inplace=True)
all_monthly_financials.head()

,Respondent,Year,Month,Sales,Total income,Purchases,Salary expense,Utility expense,Miscellaneous expense,Rent expense,...,Land,Informal loans given,Loans to employees,Total assets,Ap,Informal loan,Long term loan,Short term loan,Total liabilities,Equity
0,4425.0,2021,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,200000.0,0.0,1004400.0,0.0,0.0,0.0,0.0,0.0,1004400.0
1,4425.0,2021,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,200000.0,0.0,1404400.0,0.0,0.0,0.0,0.0,0.0,1404400.0
2,4425.0,2021,4,19515.0,19515.0,-10416.0,0.0,-280.0,0.0,0.0,...,0.0,200000.0,0.0,1411119.0,0.0,0.0,0.0,0.0,0.0,1411119.0
3,4425.0,2021,5,30157.0,30157.0,-6874.0,-2500.0,-480.0,-7202.0,-6000.0,...,0.0,200000.0,0.0,1411223.0,0.0,0.0,0.0,0.0,0.0,1411223.0
4,4425.0,2021,6,43374.0,43374.0,-4337.0,-4000.0,0.0,0.0,-6000.0,...,0.0,200000.0,0.0,1834315.0,0.0,0.0,0.0,0.0,0.0,1834315.0


In [27]:
all_monthly_financials['Class'] = all_monthly_financials['Respondent'].apply(lambda x: res_class_map[x])

In [28]:
all_monthly_financials.to_csv('all_monthly_financials_with_class.csv')

In [29]:
all_monthly_financials.head()

,Respondent,Year,Month,Sales,Total income,Purchases,Salary expense,Utility expense,Miscellaneous expense,Rent expense,...,Informal loans given,Loans to employees,Total assets,Ap,Informal loan,Long term loan,Short term loan,Total liabilities,Equity,Class
0,4425.0,2021,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,200000.0,0.0,1004400.0,0.0,0.0,0.0,0.0,0.0,1004400.0,Strong
1,4425.0,2021,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,200000.0,0.0,1404400.0,0.0,0.0,0.0,0.0,0.0,1404400.0,Strong
2,4425.0,2021,4,19515.0,19515.0,-10416.0,0.0,-280.0,0.0,0.0,...,200000.0,0.0,1411119.0,0.0,0.0,0.0,0.0,0.0,1411119.0,Strong
3,4425.0,2021,5,30157.0,30157.0,-6874.0,-2500.0,-480.0,-7202.0,-6000.0,...,200000.0,0.0,1411223.0,0.0,0.0,0.0,0.0,0.0,1411223.0,Strong
4,4425.0,2021,6,43374.0,43374.0,-4337.0,-4000.0,0.0,0.0,-6000.0,...,200000.0,0.0,1834315.0,0.0,0.0,0.0,0.0,0.0,1834315.0,Strong


In [30]:
x_cols = [col for col in all_monthly_financials.columns if col not in ['Year', 'Month', 'Respondent', 'Class']]
y_col = 'Class'

print(x_cols)
print(y_col)

['Sales', 'Total income', 'Purchases', 'Salary expense', 'Utility expense', 'Miscellaneous expense', 'Rent expense', 'Professional fees', 'Transport expense', 'Tax expense', 'Loss from theft', 'Bank fee', 'Total expense', 'Gifts given', 'Total other income and expenses', 'Net income', 'Withdrawal', 'Retained earnings', 'Cash', 'Ar', 'Equipment', 'Building', 'Live stock', 'Land', 'Informal loans given', 'Loans to employees', 'Total assets', 'Ap', 'Informal loan', 'Long term loan', 'Short term loan', 'Total liabilities', 'Equity']
Class


In [31]:
all_monthly_financials.groupby('Class')['Respondent'].count()

Class
Medium    710
Poor      282
Strong    784
Weak      132
Name: Respondent, dtype: int64

#### All monthly financial statements gave me an accuracy of 84% when predicting between Strong, medium, week and poor borrowers.

In [32]:
rand_forest3 = classification_algorithms.RandForest(all_monthly_financials, x_cols, y_col, 10)
summ3 = rand_forest3.summarize()
print(summ3['confusion_matrix'], summ3['accuracy'], sep='\n')
print('Cross validation', rand_forest3._cross_validate())

{0: 'Strong', 1: 'Poor', 2: 'Medium', 3: 'Weak'}
    Class  Strong  Poor  Medium  Weak  precision
0  Strong     183     1      14     0   0.924242
1    Poor       7    42      12     0   0.688525
2  Medium      12     2     169     0   0.923497
3    Weak       7     4       7    17   0.485714
0.8616352201257862
Cross validation 0.8392826685509613


The model isn't as good at predicting respondents with poor and weak ability to repay as it is with strong and medium. Detecting the classes "poor" and "weak" is more important but the data for the two groups is quite small. Next I will try to balance the data set and try again. 

I also have to investigate if the accuracy is because of the level of aggregation (like monthly, weekly) or the way of aggregation (like sum or mean) or because of the way the data is further processed and organized (the financial statements). It might be because of all three things but I have to understand how much each element contibutes.

In [33]:
monthly_fin_with_demo = pd.merge(all_monthly_financials, main_demo_data, how='left', on='Respondent')
monthly_fin_with_demo.head()

,Respondent,Year,Month,Sales,Total income,Purchases,Salary expense,Utility expense,Miscellaneous expense,Rent expense,...,Number of children,Marital Status,Country of Residence,Citizenship,Sector type,Number of Owners,Owner/s Gender,Number of Employees,Industry,Location in the Country
0,4425.0,2021,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
1,4425.0,2021,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
2,4425.0,2021,4,19515.0,19515.0,-10416.0,0.0,-280.0,0.0,0.0,...,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
3,4425.0,2021,5,30157.0,30157.0,-6874.0,-2500.0,-480.0,-7202.0,-6000.0,...,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
4,4425.0,2021,6,43374.0,43374.0,-4337.0,-4000.0,0.0,0.0,-6000.0,...,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa


In [34]:
x_cols2 = [col for col in monthly_fin_with_demo.columns if col not in ['Year', 'Month', 'Respondent', 'Class']]
x_cols2

['Sales',
 'Total income',
 'Purchases',
 'Salary expense',
 'Utility expense',
 'Miscellaneous expense',
 'Rent expense',
 'Professional fees',
 'Transport expense',
 'Tax expense',
 'Loss from theft',
 'Bank fee',
 'Total expense',
 'Gifts given',
 'Total other income and expenses',
 'Net income',
 'Withdrawal',
 'Retained earnings',
 'Cash',
 'Ar',
 'Equipment',
 'Building',
 'Live stock',
 'Land',
 'Informal loans given',
 'Loans to employees',
 'Total assets',
 'Ap',
 'Informal loan',
 'Long term loan',
 'Short term loan',
 'Total liabilities',
 'Equity',
 'Gender',
 'Age',
 'Number of children',
 'Marital Status',
 'Country of Residence',
 'Citizenship',
 'Sector type',
 'Number of Owners',
 'Owner/s Gender',
 'Number of Employees',
 'Industry',
 'Location in the Country']

In [35]:
rand_forest4 = classification_algorithms.RandForest(monthly_fin_with_demo, x_cols2, y_col, 10)
summ4 = rand_forest4.summarize()
print(summ4['confusion_matrix'], '-----Accuracy-----', summ4['accuracy'], sep='\n')
print('Cross validation', rand_forest4._cross_validate())

{0: 'Strong', 1: 'Poor', 2: 'Medium', 3: 'Weak'}
    Class  Strong  Poor  Medium  Weak  precision
0  Strong     198     0       0     0   1.000000
1    Poor       2    53       6     0   0.868852
2  Medium       0     0     183     0   1.000000
3    Weak       0     0       1    34   0.971429
-----Accuracy-----
0.9811320754716981
Cross validation 0.9811432469969056


That is very suspecious. Checking if the added accuracy is because of the repeated demographic data or not.

In [36]:
x_demo = [col for col in demo_cols if col != 'Respondent ID']
x_demo

['Gender',
 'Age',
 'Number of children',
 'Marital Status',
 'Country of Residence',
 'Citizenship',
 'Sector type',
 'Number of Owners',
 'Owner/s Gender',
 'Number of Employees',
 'Industry',
 'Location in the Country']

In [37]:
rand_forest5 = classification_algorithms.RandForest(monthly_fin_with_demo, x_demo, y_col, 10)
summ5 = rand_forest5.summarize()
print(summ5['confusion_matrix'], '-----Accuracy-----', summ5['accuracy'], sep='\n')
print('Cross validation', rand_forest5._cross_validate())

{0: 'Strong', 1: 'Poor', 2: 'Medium', 3: 'Weak'}
    Class  Strong  Poor  Medium  Weak  precision
0  Strong     198     0       0     0   1.000000
1    Poor       1    60       0     0   0.983607
2  Medium       0     0     183     0   1.000000
3    Weak       0     0       0    35   1.000000
-----Accuracy-----
0.9979035639412998
Cross validation 0.9958139421554056


Conclusion: The repeated demographic data is how the prediction got better. This is wrong.

### Trying average monthly financial statements for the prediction

In [38]:
average_monthly_financials = all_monthly_financials.groupby(by='Respondent')[x_cols].mean().reset_index()
average_monthly_financials['Class'] = average_monthly_financials['Respondent'].apply(lambda x: res_class_map[x])
average_monthly_financials

,Respondent,Sales,Total income,Purchases,Salary expense,Utility expense,Miscellaneous expense,Rent expense,Professional fees,Transport expense,...,Informal loans given,Loans to employees,Total assets,Ap,Informal loan,Long term loan,Short term loan,Total liabilities,Equity,Class
0,4425.0,31702.352941,31702.352941,-5582.294118,-2470.588235,-196.411765,-2103.058824,-1411.764706,-105.588235,-35.294118,...,202058.823529,0.0,1.857142e+06,398.235294,2764.705882,0.0,0.0,3162.941176,1.853979e+06,Strong
1,4429.0,144073.071429,144073.071429,-96530.828571,-13395.714286,-85.714286,-1571.428571,0.000000,0.000000,-507.142857,...,73571.428571,0.0,2.663410e+06,0.000000,-13928.571429,0.0,0.0,-13928.571429,2.677339e+06,Strong
2,4431.0,1750.000000,1750.000000,-5375.000000,0.000000,-250.000000,-62.500000,-202.500000,0.000000,-275.000000,...,50000.000000,0.0,6.321600e+05,0.000000,0.000000,0.0,0.0,0.000000,6.321600e+05,Poor
3,4432.0,2150.000000,2150.000000,0.000000,-780.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,1.763800e+05,0.000000,0.000000,0.0,0.0,0.000000,1.763800e+05,Poor
4,4433.0,24984.615385,24984.615385,-66.153846,-8565.384615,-113.882308,-421.692308,-2030.769231,0.000000,-160.384615,...,0.000000,0.0,5.086501e+05,0.000000,0.000000,0.0,0.0,0.000000,5.086501e+05,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,4862.0,53412.500000,53412.500000,-9458.333333,-3070.833333,0.000000,-300.000000,-5000.000000,0.000000,-45.833333,...,0.000000,0.0,4.214792e+05,0.000000,0.000000,0.0,0.0,0.000000,4.214792e+05,Strong
154,4864.0,46415.384615,46415.384615,-16461.538462,-4280.769231,-46.153846,0.000000,-615.384615,0.000000,0.000000,...,5500.000000,0.0,5.029231e+05,0.000000,0.000000,0.0,0.0,0.000000,5.029231e+05,Poor
155,5104.0,10903.000000,10903.000000,-57.142857,-857.142857,-516.285714,-646.428571,-1785.714286,0.000000,-83.214286,...,142.857143,0.0,3.862546e+05,0.000000,0.000000,0.0,0.0,0.000000,3.862546e+05,Weak
156,5127.0,35609.676000,35609.676000,-11248.200000,0.000000,0.000000,-911.600000,0.000000,0.000000,-371.000000,...,261000.000000,0.0,-3.565750e+05,0.000000,-322330.000000,-315000.0,0.0,-637330.000000,2.807550e+05,Poor


In [39]:
rand_forest6 = classification_algorithms.RandForest(average_monthly_financials, x_cols, y_col, 10)
summ6 = rand_forest6.summarize()
print(summ6['confusion_matrix'], '-----Accuracy-----', summ6['accuracy'], sep='\n')
print('Cross validation', rand_forest6._cross_validate())

{0: 'Strong', 1: 'Poor', 2: 'Medium', 3: 'Weak'}
    Class  Strong  Poor  Medium  Weak  precision
0  Strong      10     0       4     0   0.714286
1    Poor       2     3       1     0   0.500000
2  Medium       5     0      10     0   0.666667
3    Weak       2     1       2     0   0.000000
-----Accuracy-----
0.575
Cross validation 0.41557971014492756


Adding demographic data to the dataset to see if it helps.

In [40]:
average_mon_demo = pd.merge(average_monthly_financials, main_demo_data, how='left', on='Respondent')
average_mon_demo.head()

,Respondent,Sales,Total income,Purchases,Salary expense,Utility expense,Miscellaneous expense,Rent expense,Professional fees,Transport expense,...,Number of children,Marital Status,Country of Residence,Citizenship,Sector type,Number of Owners,Owner/s Gender,Number of Employees,Industry,Location in the Country
0,4425.0,31702.352941,31702.352941,-5582.294118,-2470.588235,-196.411765,-2103.058824,-1411.764706,-105.588235,-35.294118,...,3,Married,Ethiopia,Ethiopia,AD,1,All men,4,Other,Diredawa
1,4429.0,144073.071429,144073.071429,-96530.828571,-13395.714286,-85.714286,-1571.428571,0.000000,0.000000,-507.142857,...,0,Single,Ethiopia,Ethiopia,EE-01,2,All men,18,Light Manu,Addis Ababa
2,4431.0,1750.000000,1750.000000,-5375.000000,0.000000,-250.000000,-62.500000,-202.500000,0.000000,-275.000000,...,0,Single,Ethiopia,Ethiopia,Garment production,4 or more,Both men and women,25,Light Manu,Addis Ababa
3,4432.0,2150.000000,2150.000000,0.000000,-780.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2,Married,Ethiopia,Ethiopia,U-03,1,All men,5,Other,Diredawa
4,4433.0,24984.615385,24984.615385,-66.153846,-8565.384615,-113.882308,-421.692308,-2030.769231,0.000000,-160.384615,...,1,Widowed,Ethiopia,Ethiopia,X,3,Both men and women,12,Other,Harar


In [41]:
rand_forest7 = classification_algorithms.RandForest(average_mon_demo, x_cols, y_col, 10)
summ7 = rand_forest7.summarize()
print(summ7['confusion_matrix'], '-----Accuracy-----', summ7['accuracy'], sep='\n')
print('Cross validation', rand_forest7._cross_validate())

{0: 'Strong', 1: 'Poor', 2: 'Medium', 3: 'Weak'}
    Class  Strong  Poor  Medium  Weak  precision
0  Strong       8     1       5     0   0.571429
1    Poor       2     3       1     0   0.500000
2  Medium       7     1       7     0   0.466667
3    Weak       3     1       1     0   0.000000
-----Accuracy-----
0.45
Cross validation 0.4333333333333333
